We loaded the data while specifying the encoding since it produced an error

In [158]:
import pandas as pd
import numpy as np

df = pd.read_csv("./Data/AviationData.csv", encoding='ISO-8859-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

## Columns to remove

We decided on deleting columns that don't hold much significance in determining which aircraft are safe to invest in.
The ones we decided on initially are:  
Airport.Code  
Airport.Name  
Latitude   
Longitude
Total.Minor.Injuries  
Total.Uninjured    
Publication.Date    
Total.Fatal.Injuries  
Schedule  
Broad.phase.of.flight
Accident.Number  
Event.Id  
FAR.Description  
Registration.Number	 
Report.Status  


In [159]:
columns_to_drop = ['Airport.Code',  
'Airport.Name',  
'Latitude',   
'Longitude',  
'Total.Minor.Injuries',
'Total.Uninjured',
'Publication.Date', 
'Broad.phase.of.flight',
'Total.Fatal.Injuries',
'Schedule',  
'FAR.Description',  
'Registration.Number',
'Accident.Number',
'Event.Id',
'Report.Status']
df = df

df.drop(columns = columns_to_drop, inplace=True)

## Reformatting text

We found that some columns have text with differing punctuation or capitalization, causing them to be recognized as separate entries. 

For example, in the 'Make' category, Cessna and CESSNA are considered different categories.   
In order to fix this, we will go through and strip leading and trailing whitespace and make them similar case.  

Another column we did this to was the model column

In [160]:
#Format Make column, some vals are floats so cast to string first
df['Make'] = df['Make'].map(lambda make: str(make).strip().title())

#Format model column
df['Model'] = df['Model'].map(lambda model: str(model).strip().title() if pd.notnull(model) else model)

Another change we are making is to separate the Location column into two separate columns, since the capitalization is not consistent and having the state abbreviation on its own will make it easier to access

In [161]:
#Format Weather Conditions so that all unknown vals are just changed to nan
#df['Weather.Condition'] = df['Weather.Condition'].map(lambda weather: str(weather).upper() if pd.notnull(weather) else weather)
df['Weather.Condition'] = df['Weather.Condition'].replace({'Unk': np.NaN, 'UNK': np.NaN})

We decided that splitting the location column into two separate columns would make comparing rows easier, so the locations where split on the comma and separated

In [162]:
#Split Location into two columns and drop the original, then strip and case the new columns
df[['City', 'State']] = df['Location'].str.split(',', n=1, expand=True)
df.drop(columns = ['Location'], inplace=True)
df['City'] = df['City'].map(lambda city: str(city).strip().title() if pd.notnull(city) else city)
df['State'] = df['State'].map(lambda state: str(state).strip().upper() if pd.notnull(state) else state)

The injury severity column had a more drastic change to it. While inspecting the column, we noticed that the Injury.Severity column somewhat combined the Total.Fatal.Injuries and the Total.Serious.Injuries. If the Total.Fatal.Injuries column was non zero, then Injury.Severity would reflect that with Fatal(#) where # would hold an int. If not, then Injury.Severity would have some text describing it, such as 'Minor' or 'Non-Fatal'. Because of that, we decided to remove the Fatal text and instead just keep the number since it would be easier to check if the column was an int.

In [163]:
#Format Injury Severity where Fatal is removed so that only the integer exists
df['Injury.Severity'] = df['Injury.Severity'].map(lambda injured:''.join(x for x in str(injured) if x.isdigit()) if (any(i.isdigit() for i in str(injured)) & pd.notnull(injured)) else injured)

## Dropping Rows

If we look at the Aircraft.Category column, we see that there are a bunch of aircraft that aren't even aircraft. We decided to not keep these and drop everthing that isn't either an airplane or helicopter

In [164]:
df['Aircraft.Category'].value_counts()

Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
ULTR                     1
Rocket                   1
Name: Aircraft.Category, dtype: int64

In [165]:
#Drop everything that isn't an airplane or helicopter or if null
df = df[(df['Aircraft.Category'] == 'Airplane') | (df['Aircraft.Category'] == 'Helicopter') | (df['Aircraft.Category'].isnull())]


## Cleaned dataset

Here we can see now that there are many less columns and the column entries have been formated to make calculations easier. Then we can write to another csv for easy access.

In [166]:
df.to_csv('./Data/CleanedAviationData.csv', encoding='utf-8')